# Classifying newswires

## Loading Reuters dataset

In [2]:
from keras.datasets import reuters

(train_data, train_labels), (test_data, test_labels) = reuters.load_data(num_words = 10000)

attribute of dataset

In [ ]:
print(len(train_data))
print(len(test_data))
print(train_data[0])
print(train_labels[0])

decode newswire back to text
 ```
# word_index is a dictionary mapping words to an integer index
word_index = imdb.get_word_index()
# We reverse it, mapping integer indices to words
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
# We decode the review; note that our indices were offset by 3
# because 0, 1 and 2 are reserved indices for "padding", "start of sequence", and "unknown".
decoded_review = ' '.join([reverse_word_index.get(i - 3, '?') for i in train_data[0]])
# show
decoded_review
```



In [25]:
word_index = reuters.get_word_index()
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
decoded_newswire = ' '.join([reverse_word_index.get(i - 3, '?') for i in
train_data[0]])
print(decoded_newswire)

? ? ? said as a result of its december acquisition of space co it expects earnings per share in 1987 of 1 15 to 1 30 dlrs per share up from 70 cts in 1986 the company said pretax net should rise to nine to 10 mln dlrs from six mln dlrs in 1986 and rental operation revenues to 19 to 22 mln dlrs from 12 5 mln dlrs it said cash flow per share this year should be 2 50 to three dlrs reuter 3


## Preparing the data

vectorize data

Note:
```
numpy.zeros(shape, dtype=float, order='C')
```

In [3]:
import numpy as np

def vectorize_sequences(sequences, dimension = 10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return results

x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)

One-hot method to vectorize labels.

In [4]:
def vectorize_labels(labels, dimension = 46):
    results = np.zeros((len(labels), dimension))
    for i, label in enumerate(labels):
        results[i, label] = 1.
    return results

one_hot_y_train = vectorize_labels(train_labels)
one_hot_y_test = vectorize_labels(test_labels)

There is a bulit-in way to do above in Keras.

In [28]:
from keras.utils.np_utils import to_categorical
ont_hot_y_train = to_categorical(train_labels)
one_hot_y_test = to_categorical(test_labels)

## Building Network

In [5]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(64, activation = 'relu', input_shape = (10000,)))
model.add(layers.Dense(64, activation = 'relu'))
model.add(layers.Dense(46, activation = 'softmax'))

model conpile

In [30]:
model.compile(optimizer = 'rmsprop',
              loss = 'categorical_crossentropy',
              metrics = ['accuracy'])

## validating your approach

Setting aside a validation set

In [7]:
x_val = x_train[:1000]
partial_x_train = x_train[1000:]
y_val = one_hot_y_train[:1000]
partial_y_train = one_hot_y_train[1000:]

## training models

In [ ]:
history = model.fit(partial_x_train, 
                    partial_y_train,
                    epochs = 20, 
                    batch_size = 512,
                    validation_data=(x_val, y_val))

plotting the training and validation loss

In [ ]:
import matplotlib.pyplot as plt
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

plotting the traing and validation acc

In [ ]:
plt.clf()
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

```
The network begins to overfit after nine epochs. Let’s train a new network from
scratch for nine epochs and then evaluate it on the test set.
```

In [ ]:
model = models.Sequential()
model.add(layers.Dense(64, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(46, activation='softmax'))
model.compile(optimizer='rmsprop',
loss='categorical_crossentropy',
metrics=['accuracy'])
model.fit(partial_x_train,
          partial_y_train,
          epochs=9,
          batch_size=512,
          validation_data=(x_val, y_val))
results = model.evaluate(x_test, one_hot_y_test)

a random baseline

In [44]:
import copy
test_labels_copy = copy.copy(test_labels)
np.random.shuffle(test_labels_copy)
hits_array = np.array(test_labels) == np.array(test_labels_copy)
float(np.sum(hits_array)) / len(test_labels)

0.19723953695458593

genrate predictions on new data.

In [ ]:
predictions = model.predict(x_test)
print(predictions, predictions[0].shape, np.sum(predictions[0]), np.argmax(predictions[0]))

## A different way to handle labels and loss.

## A model with an information bottleneck.

In [8]:
model = models.Sequential()
model.add(layers.Dense(64, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(4, activation='relu'))
model.add(layers.Dense(46, activation='softmax'))
model.compile(optimizer='rmsprop',
loss='categorical_crossentropy',
metrics=['accuracy'])
model.fit(partial_x_train,
partial_y_train,
epochs=20,
batch_size=128,
validation_data=(x_val, y_val))

Epoch 1/20
63/63 [==============================] - 1s 7ms/step - loss: 3.3727 - accuracy: 0.1267 - val_loss: 2.8595 - val_accuracy: 0.4100
Epoch 2/20
63/63 [==============================] - 0s 4ms/step - loss: 2.3001 - accuracy: 0.4540 - val_loss: 1.8846 - val_accuracy: 0.4670
Epoch 3/20
63/63 [==============================] - 0s 4ms/step - loss: 1.5596 - accuracy: 0.6061 - val_loss: 1.4739 - val_accuracy: 0.6660
Epoch 4/20
63/63 [==============================] - 0s 4ms/step - loss: 1.2335 - accuracy: 0.7083 - val_loss: 1.3376 - val_accuracy: 0.6950
Epoch 5/20
63/63 [==============================] - 0s 4ms/step - loss: 1.0684 - accuracy: 0.7425 - val_loss: 1.2723 - val_accuracy: 0.7060
Epoch 6/20
63/63 [==============================] - 0s 4ms/step - loss: 0.9628 - accuracy: 0.7627 - val_loss: 1.2551 - val_accuracy: 0.7210
Epoch 7/20
63/63 [==============================] - 0s 4ms/step - loss: 0.8826 - accuracy: 0.7858 - val_loss: 1.2420 - val_accuracy: 0.7240
Epoch 8/20
63/63 [==